### Laboratorio de Escalonamento ###

Neste laboratório, iremos simular o funcionamento de algoritmos de escalonamento para entender melhor seu funcionamento.

Na célula abaixo, temos uma classe Processo, que tem as informações de execução, e uma classe Round Robin de exemplo de escalonamento:

In [2]:
#
#Luis de Gonzaga Mota Pinto Filho - 21753684
#Evaldo Patrik dos Santos Cardoso - 21453640
#

import numpy as np
import random
import MergeSort as sort

class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada):
        self.nome = pnome
        self.io = pio
        self.tam = ptam
        self.prio = prioridade
        self.chegada = 0
        self.resposta = -1

    def roda(self,quantum):
        if(random.randint(1,100)<self.io):
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True
            
            
        if(self.tam<quantum):
            quantum = self.tam
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        return quantum, False
    


class round_robin(object):
    def __init__(self,vprontos):
        self.prontos = vprontos

    def pronto(self,Processo):
        self.prontos.append(Processo)
        
    def proximo(self):
        if(len(self.prontos)==0):
            return None
        p = self.prontos[0]
        del self.prontos[0]
        return p
    
class lottery_sched(object):
    def __init__(self,vprontos):
        self.prontos = sort.merge_sort(vprontos)
    
    def pronto(self,Processo):
        self.prontos.append(Processo)
        
    def proximo(self):
        if(len(self.prontos)==0):
            return None
        tickets = 100
        lottery = random.randint(1,tickets)
        for i in range(len(self.prontos)):
            if(lottery<=self.prontos[i].prio):
                p = self.prontos[i]
                del self.prontos[i]
                return p
            

class fila(object):
    def __init__(self,quantum,prio):
        self.quantum = quantum
        self.prio = prio
        self.prontos = []
        
    def enfileira(self,processo):
        self.prontos.append(processo)
            

class mlfq(object):
    def __init__(self,vprontos,numeroFilas,vquantum,s):
        self.prontos = sort.merge_sort(vprontos)
        self.filas = []
        self.tempoBoost = s
        self.comp = 0
        for i in range(numeroFilas):
            f = fila(vquantum[i],i+1)
            self.filas.append()
        for procc in self.prontos:
            filas[procc.prio-1].enfileira(procc)
        
        
    def pronto(self,Processo):
        self.filas[Processo.prio-1].enfileira(Processo)
        
    def proximo(self,tempo):
        if((tempo - self.comp) != self.tempoBoost):
            for fil in self.filas:
                if(len(fil)==0):
                    continue
                else:
                    p = fil[0]
                    del fil[0]
                    return p
        elif((tempo - self.comp) == self.tempoBoost):
            self.comp = tempo
            for fil in self.filas:
                for p in fil:
                    p.prio = 1
                    
                
            return self.filas[0][0]
                    

Na célula abaixo, são criados quatro processos completamente CPU-Bound que precisam de 3 timeslices para rodar.

O valor de E/S é um número entre 0 e 100 indicando quantos porcento de chance o processo tem de fazer E/S durante seu tempo na CPU

In [10]:
nprocs = 10
nomes = ['A','B','C','D','a','s','g','gr','sdg','adg']
chanceIo = [0,0,0,0,0,0,0,0,0,0] #Valor de zero a cem, chance de ser entrada e saida
prioridade = [10,20,50,20,0,0,0,0,0,0]
tamanho = np.array([3,3,3,3,3,3,3,3,3,3])


total = tamanho.sum()

procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i],chanceIo[i],tamanho[i],prioridade[i],0))

Na célula abaixo, temos uma simulação do funcionamento de um escalonador de processos. As duas configurações importantes aqui são o valor do quantum padrão (que pode ser dinamico em algoritmos mais complexos, e quantos timeslices um processo que faz e/s passa bloqueado.

Percebam que na terceira linha é instanciado o escalonador (neste caso, um round_robin). Isto foi feito assim para ser simples trocar o escalonador e repetir a simulação, bastando criar uma classe com os métodos pronto, proximo e construtor e alterar esta linha.

In [11]:
quantum = 2
tempoBloq = 1

escalonador = round_robin(procs)
bloqueados = []

tempo = 0

tempoResposta = np.array([])
tempoExecucao = np.array([])

random.seed(0)

while total>0:
    p = escalonador.proximo()
    if(p is not None):
        rodou, fezIo = p.roda(quantum)
        if(fezIo and p.tam>0):
            bloqueados.append([p,tempoBloq+1]) #Adiciona o processo que fez e/s aos bloqueados
        elif(p.tam>0):
            escalonador.pronto(p)
        total-=rodou
        if(p.resposta == -1):
            tempoResposta = np.append(tempoResposta,tempo)
            p.resposta = tempo
        tempo+=rodou
        if(p.tam==0):
            tempoExecucao = np.append(tempoExecucao,(tempo-p.resposta))
    else:
        #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
        tempo+=1
    if(len(bloqueados)>0):
        for i in bloqueados:
            i[1]-=1
            if(i[1]==0):
                escalonador.pronto(i[0])
                del i  
                
print("tempo de resposta medio: ",tempoResposta.sum()/nprocs)
print("tempo de execucao medio: ",tempoExecucao.sum()/nprocs)
print("Unfairness: ",tempoExecucao[0]/tempoExecucao[1])

A  rodou por  2  timeslice, faltam  1
B  rodou por  2  timeslice, faltam  1
C  rodou por  2  timeslice, faltam  1
D  rodou por  2  timeslice, faltam  1
a  rodou por  2  timeslice, faltam  1
s  rodou por  2  timeslice, faltam  1
g  rodou por  2  timeslice, faltam  1
gr  rodou por  2  timeslice, faltam  1
sdg  rodou por  2  timeslice, faltam  1
adg  rodou por  2  timeslice, faltam  1
A  rodou por  1  timeslice, faltam  0
B  rodou por  1  timeslice, faltam  0
C  rodou por  1  timeslice, faltam  0
D  rodou por  1  timeslice, faltam  0
a  rodou por  1  timeslice, faltam  0
s  rodou por  1  timeslice, faltam  0
g  rodou por  1  timeslice, faltam  0
gr  rodou por  1  timeslice, faltam  0
sdg  rodou por  1  timeslice, faltam  0
adg  rodou por  1  timeslice, faltam  0
tempo de resposta medio:  9.0
tempo de execucao medio:  16.5
Unfairness:  1.05


### Laboratório: ###


1 - Altere o simulador acima para calcular também o tempo de execucao medio e tempo de resposta 

2 - Implemente o escalonador por loteria e verifique seu funcionamento com diferentes configurações de números de tickets (Use o parametro Prio do processo como número de tickets).

3 - Implemente o MLFQ, conforme os slides. Número de filas, quantum por fila e tempo para boost devem ser escolhidos por vocês (e de preferencia ser parametrizável)


Após suas implementações estarem bem azeitadas, vocês podem testá-las neste outro simulador mais configurável:

In [32]:
quantum = 2
tempoBloq = 1

escalonador = lottery_sched(procs)
bloqueados = []


maximo = 10
chanceNovoProcesso = 60
chanceIo = 30
minTime = 4
maxTime = 10

tempoResposta = np.array([])
tempoExecucao = np.array([])


contaProc = 1
tempo = 0
while tempo<maximo or len(escalonador.prontos)>0:
    
    #Novo processo tem chanceProcesso% de chance surgir
    if(tempo<maximo and random.randint(1,100)<chanceNovoProcesso):
        p = Processo('P'+str(contaProc),random.randint(1,chanceIo),random.randint(minTime,maxTime),0,tempo)
        print("Processo",p.nome," criado no tempo",tempo)
        escalonador.pronto(p)
        contaProc+=1
        

    p = escalonador.proximo()
    if(p is not None):
        rodou, fezio = p.roda(quantum)
        if(fezio and p.tam>0):
            bloqueados.append([p,tempoBloq+1]) #Adiciona o processo que fez e/s aos bloqueados
        elif(p.tam>0):
            escalonador.pronto(p)
        total-=rodou
        if(p.resposta == -1):
            tempoResposta = np.append(tempoResposta,tempo)
            p.resposta = tempo
        tempo+=rodou
        if(p.tam==0):
            tempoExecucao = np.append(tempoExecucao,(tempo-p.resposta))
    else:
        #Reduz o tempo de todos os bloqueados em uma unidade
        tempo+=1
    if(len(bloqueados)>0):
        for i in bloqueados:
            i[1]-=1
            if(i[1]==0):
                escalonador.pronto(i[0])
                del i  
                
print("tempo de resposta medio: ",tempoResposta.sum()/nprocs)
print("tempo de execucao medio: ",tempoExecucao.sum()/nprocs)

Processo P1  criado no tempo 0
Processo P2  criado no tempo 1
Processo P3  criado no tempo 2
Processo P4  criado no tempo 5
Processo P5  criado no tempo 6
A  rodou por  2  timeslice, faltam  4
B  rodou por  2  timeslice, faltam  6
P1  rodou por  2  timeslice, faltam  2
P2  rodou por  2  timeslice, faltam  6
P3  rodou por  2  timeslice, faltam  4
P4  fez e/s, falta  7
P5  fez e/s, falta  7
A  rodou por  2  timeslice, faltam  2
B  rodou por  2  timeslice, faltam  4
P1  rodou por  2  timeslice, faltam  0
P2  fez e/s, falta  5
P3  rodou por  2  timeslice, faltam  2
P4  fez e/s, falta  6
P5  rodou por  2  timeslice, faltam  5
A  rodou por  2  timeslice, faltam  0
B  rodou por  2  timeslice, faltam  2
P2  rodou por  2  timeslice, faltam  3
P3  rodou por  2  timeslice, faltam  0
P4  rodou por  2  timeslice, faltam  4
P5  rodou por  2  timeslice, faltam  3
B  rodou por  2  timeslice, faltam  0
P2  rodou por  2  timeslice, faltam  1
P4  rodou por  2  timeslice, faltam  2
P5  rodou por  2  times

## Responda ##

#### Sobre Loteria: ####

1 - Rode a simulação mais simples com apenas dois processos, um com 100 tickets e outro com 1, com tempo 100 para terminar. O que aconteceu? O segundo processo conseguiu rodar alguma vez? Testando com outras sementes o resultado se manteve?

2 - Rode a simulação simples com dois processos com 100 tiquets e 100 timeslices. Calcule a Unfairness para quantum=2. Repita a operação para quantum 10, 20, 50 e 100. O que aconteceu?

3 - Rode o simulador maior com três tipos de processo possíveis: um com 10 tickets, um com 20 e um com 50. O que aconteceu com os que receberam menos tickets? Eles tiveram chance de rodar?

In [ ]:
# 1 - o primeiro processo sempre roda todo antes do segundo ir para a cpu pela primeira vez
# 2 - a métrica unfairness não sofreu drásticas variações com a mudança do quantum. a métrica permaneceu
#     proxima de 1 devido ao tamanho dos processos
# 3 - Tiveram. Se o processo receber pelo menos 1 ticket ele irá rodar.

#### Sobre MLFQ ####

1 - Rode o MLFQ no segundo simulador com duas filas e sem boost. O que aconteceu? 

2 - Quais configurações de parâmetros transformam o MLFQ no Round Robin?

3 - Rodando no primeiro simulador, faça 1 dos quatro processos ser 100\% CPU-Bound e os outros três 60\% I/O Bound. Use a configuração com 3 filas, com 5, 3 e 1 timeslices para as fila em ordem de priorida. De quanto tempo deve ser o Boost para que o processo CPU-Bound não sofra starvation e tenha CPU 1 vez a cada 20 timeslices?

In [ ]:
# 1 - os processos dados inicialmente demoram muito para entrarem na CPU o que fica pior
#     aumentando o numero de processos novos que chegam
# 2 - o parametro S que é o tempo para o boost. Com este parametro muito pequeno, todos
#     estarão em prioridade alta o tempo todo, fazendo o MLFQ se comportar como o round
#     robin
#